# Binary Graph Classification
#### Done on the smallest class 457<br>using the vulnerable and safe file of this class only

In [1]:
import pandas as pd
import numpy as np
import json
import torch
import os
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, Sequential, Linear, global_mean_pool
import pickle
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
import torch.nn.functional as F




from sklearn.model_selection import train_test_split

from gcnHelpers import *

# Training on Binary class
#### please put the folders of the CVE's in the script's directory<br>then inside each folder put 2 subfolders, one for the nodes and the other for edges
##### also for the safe class, name the folder 457_SAFE, and inside it nodes_457_SAFE and edges_457_SAFE

In [2]:

#List containing all input data of type Data (Pytorch geometric datastructure that holds a single graph)
allData = []

torch.cuda.empty_cache()

cve = '78'
print(f"-----------LOADING CVE {cve} Vulnerable------------------")


pathToEdges = f"D:/ClassWork/Guardista/4-Features_Extraction/{cve}/edges_{cve}"

with open (f'{cve}/features_matrices/features_matrices_'+cve+'.npy', 'rb') as f:
    features_matrices_list = np.load(f,  allow_pickle=True)

stop_after = len(features_matrices_list)
print(stop_after)

# with open(f'{cve}/nodes_targets/nodes_targets_'+cve+'.npy', 'rb') as f:
#     nodes_targets_list = np.load(f,  allow_pickle=True)


print(f"num Graphs : {features_matrices_list.shape[0]}")


# Reading the adjacency list of each graph
adj_Lists = []
for i, filename in enumerate(os.listdir(pathToEdges)):
    if i == stop_after:
        break
    try:
        f = os.path.join(pathToEdges, filename)
        df = pd.read_csv(f, header=None)
        df.dropna(inplace=True)             # <---- here is the dropping
        adj_Lists.append(df)
    except:
        print('something wrong')

        
    
print(len(adj_Lists))

numVulnNodes = 0


    

numNodes = 0
# Reading the features of each node
node_Features_List = []
for node_feature in features_matrices_list:
    n = np.array(node_feature,dtype='int64')
    numNodes+= n.shape[0]
    node_Features_List.append(torch.tensor(n, dtype=torch.float))


assert (len(adj_Lists) == len(node_Features_List))


# Convert the edge list to use 0-based indices
adj_Tensors = []
for adj in adj_Lists:
    # the documentation strictly says to convert the adjacency list to a contiguous list
    adj_Tensors.append(torch.as_tensor(adj.to_numpy(), dtype=torch.long).t().contiguous())   


#Iterate over each graph, make a Data object, then append to all our dataset
cntCorruptData=0
for i, adj in enumerate(adj_Tensors):
    d = Data(x=node_Features_List[i], edge_index=adj, y=torch.as_tensor([1]))  # <--- from the documentation.
    try:
        d.validate(raise_on_error=True)     # <--------- this line makes sure each input graph strictly follows the correct rules, to evade errors
    except:
        #print('DIMENSION ERROR')
        #print(f"We have features for {len(node_Features_List[i])} Nodes ")
        #print(f"But the adjacency list contains {max(set(np.array(adj[0,:])))} Unique Nodes")
        cntCorruptData +=1
        continue
    allData.append(d)


print(f"number of corrupted files due to missing node features for certain nodes = {cntCorruptData}")
   

-----------LOADING CVE 78 Vulnerable------------------
1500
num Graphs : 1500
1500
number of corrupted files due to missing node features for certain nodes = 622


### Loading the 457 SAFE dataset

In [3]:
print(f"-----------LOADING CVE {cve} SAFE------------------")

pathToEdges = f"D:/ClassWork/Guardista/4-Features_Extraction/{cve}_safe/edges_{cve}_safe"

with open (f'{cve}_safe/features_matrices/features_matrices_'+cve+'_safe.npy', 'rb') as f:
    features_matrices_list = np.load(f,  allow_pickle=True)

stop_after = len(features_matrices_list)
print(stop_after)

# with open(f'{cve}/nodes_targets/nodes_targets_'+cve+'.npy', 'rb') as f:
#     nodes_targets_list = np.load(f,  allow_pickle=True)


print(f"num Graphs : {features_matrices_list.shape[0]}")


# Reading the adjacency list of each graph
adj_Lists = []
for i, filename in enumerate(os.listdir(pathToEdges)):
    if i == stop_after:
        break
    try:
        f = os.path.join(pathToEdges, filename)
        df = pd.read_csv(f, header=None)
        df.dropna(inplace=True)             # <---- here is the dropping
        adj_Lists.append(df)
    except:
        print('something wrong')

        
    
print(len(adj_Lists))

numVulnNodes = 0


# Reading the label of each node
#Targets_List = []

# for target in nodes_targets_list:
#     numVulnNodes += sum(target)
#     Targets_List.append(1)

    

numNodes = 0
# Reading the features of each node
node_Features_List = []
for node_feature in features_matrices_list:
    n = np.array(node_feature,dtype='int64')
    numNodes+= n.shape[0]
    node_Features_List.append(torch.tensor(n, dtype=torch.float))


assert (len(adj_Lists) == len(node_Features_List))


# Convert the edge list to use 0-based indices
adj_Tensors = []
for adj in adj_Lists:
    # the documentation strictly says to convert the adjacency list to a contiguous list
    adj_Tensors.append(torch.as_tensor(adj.to_numpy(), dtype=torch.long).t().contiguous())   


#Iterate over each graph, make a Data object, then append to all our dataset
cntCorruptData=0
for i, adj in enumerate(adj_Tensors):
    d = Data(x=node_Features_List[i], edge_index=adj, y=torch.as_tensor([0]))  # <--- from the documentation.
    try:
        d.validate(raise_on_error=True)     # <--------- this line makes sure each input graph strictly follows the correct rules, to evade errors
    except:
        #print('DIMENSION ERROR')
        #print(f"We have features for {len(node_Features_List[i])} Nodes ")
        #print(f"But the adjacency list contains {max(set(np.array(adj[0,:])))} Unique Nodes")
        cntCorruptData +=1
        continue
    allData.append(d)


print(f"number of corrupted files due to missing node features for certain nodes = {cntCorruptData}")

-----------LOADING CVE 78 SAFE------------------
1500
num Graphs : 1500
1500
number of corrupted files due to missing node features for certain nodes = 0


### Train-Test-Split and data loader

In [4]:

# Train Test Split
allData_train, allData_test = train_test_split(allData, test_size=0.3, shuffle=True)
allData_train, allData_val = train_test_split(allData_train, test_size=0.1, shuffle=True)


unique_classes = set()

for d in allData_train:
    unique_classes.add(int(d.y.numpy()))

print(f"Unique classes {unique_classes}")


#Our Data Loader
batch_size = 5
loader = DataLoader(allData_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(allData_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(allData_test, batch_size=batch_size, shuffle=True)

Unique classes {0, 1}


# GCN Class

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

numClasses = len(list(unique_classes))

model = GCN(hidden_channels=128, numClasses=2, numFeatures=133).to(device=device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-7)
#weights = torch.tensor(np.array([2.1, 1.1]), dtype=torch.float)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(1, 21):
    train(model, loader, device, optimizer, criterion)
    train_acc = test(model, loader, device)
    val_acc = test(model, val_loader, device)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Validation Acc: {val_acc:.4f}')

# Saving the model
with open('GCN_binary.pkl', 'wb') as f:
    pickle.dump(model.cpu(), f)

GCN(
  (conv1): GCNConv(133, 128)
  (conv2): GCNConv(128, 128)
  (conv3): GCNConv(128, 128)
  (conv4): GCNConv(128, 128)
  (conv5): GCNConv(128, 128)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (lin): Linear(in_features=128, out_features=2, bias=True)
)
Epoch: 001, Train Acc: 0.8391, Validation Acc: 0.9106
Epoch: 002, Train Acc: 0.8391, Validation Acc: 0.9106
Epoch: 003, Train Acc: 0.8436, Validation Acc: 0.9187
Epoch: 004, Train Acc: 0.8436, Validation Acc: 0.9187
Epoch: 005, Train Acc: 0.9060, Validation Acc: 0.9350
Epoch: 006, Train Acc: 0.8391, Validation Acc: 0.9106
Epoch: 007, Train Acc: 0.8436, Validation Acc: 0.9187
Epoch: 008, Train Acc: 0.9340, Validation Acc: 0.9431
Epoch: 009, Train Acc: 0.9430, Validation Acc: 0.9593
Epoch: 010, Train Acc: 0.8436, Validation Acc: 0.9187
Epoch: 011, Train Acc: 0.8436, Validation Acc: 0.9187
Epoch: 012, Train Acc: 0.8436, Validation Acc: 0.9187
Epoch: 013, Train Acc: 0.8445, Validation Acc: 0.9187
Epoch: 014, Train Acc: 0

# Testing

In [6]:
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device=device)

y_true = []
y_predicted = []
unique_classes = set()
for d in allData_test:
    y_true.append(d.y.cpu().numpy())
    
    pred,_ = inference(model, d.to(device=device), device=device)
    y_predicted.append(pred)


print(classification_report(y_true= y_true, y_pred= y_predicted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       460
           1       1.00      0.99      0.99        67

    accuracy                           1.00       527
   macro avg       1.00      0.99      1.00       527
weighted avg       1.00      1.00      1.00       527



In [7]:

pathtoUserNodes = r"D:\ClassWork\Guardista\BackEnd\new_BE\tmp\tmp8\output\CFG_UserCode/nodes".replace('\\', '/')
pathToUser_Edges = r"D:\ClassWork\Guardista\BackEnd\new_BE\tmp\tmp8\output\CFG_UserCode/edges".replace('\\', '/')
outputPath = r"D:\ClassWork\Guardista\BackEnd\new_BE\tmp\tmp8\output\CFG_UserCode/nodes".replace('\\', '/')
modelPath = 'GCN_binary.pkl'
cve = 'test'




classification, embDF = InferenceGCN (model=modelPath, pathToUser_Edges=pathToUser_Edges, outputPath=outputPath, multipleFiles='false', cve=cve,npyPath=None, pathToUser_Nodes=pathtoUserNodes)


print(classification)


number of corrupted files discarded = 0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (21694x33 and 133x128)